# Курсовой проект по дисциплине "Вычислительная математика"(XIII.11.1)

# Постановка задачи:
Здесь рассмотрена более общая задача, по сравнению с XIII.11.1 (из задачника Аристовой Е.Н., Лобанова А.И.).
Для аппроксимации уравнения $$ \frac{\partial u}{\partial t}=\frac{\partial}{\partial x }a u^\alpha\frac{\partial u}{\partial x}+q(x)$$ $$ -\alpha_1 u_x(0,t) + \beta_1 u(0,t)=f_1(t) $$ $$ \alpha_2 u_x(X,t) + \beta_2 u(X,t)=f_2(t) $$ $$ u(x,0)=g(x), $$ $$ \alpha_i \geqslant 0, \beta_i \geqslant 0, \alpha_i+\beta_i > 0, i = 1,2 $$
используется следующая разностная схема $$ \frac{y^{n+1}_m - y^{n}_{m}}{\tau} = \frac{1}{h}(k_{m+\frac{1}{2}}\frac{y^{n+1}_{m+1} - y^{n+1}_{m}}{h} - k_{m-\frac{1}{2}}\frac{y^{n+1}_{m} - y^{n+1}_{m-1}}{h}) $$

Предлагается использовать различные способы определения $ k_{m+\frac{1}{2}} $:

$ 1) \quad k_{m+\frac{1}{2}} = \frac{a}{2} ({y^{n}_{m}}^\alpha + {y^{n}_{m+1}}^\alpha)$ $$$$
$ 2) \quad k_{m+\frac{1}{2}} = a(\frac{{y^{n}_{m}} + {y^{n}_{m+1}}}{2})^\alpha$ $$$$
$ 3) \quad k_{m+\frac{1}{2}} = a(\frac{2{y^{n}_{m}} {y^{n}_{m+1}}}{{y^{n}_{m}}+{y^{n}_{m+1}}})^\alpha$ $$$$
$ 4) \quad k_{m+\frac{1}{2}} = 2a(\frac{{y^{n}_{m}}^\alpha {y^{n}_{m+1}}^\alpha}{{y^{n}_{m}}^\alpha+{y^{n}_{m+1}}^\alpha})$ $$$$

Какой из вариантов предпочтительнее? Почему не работают в окрестности фронта средние гармонические — 3) и 4)?
Реализовать схемы 1) и 2), сравнить численное решение с точным. Почему при больших значениях $\frac{\tau}{h^2}$ наблюдается отставание фронта волны в численном решении от точного значения?

# Решение

## 1.Ответы на вопросы задачи
Первым шагом было получено, что рассматриваемая явная схема является устойчивой при выполнении условия: 
$$ h^2 \geqslant \tau(k_{m+\frac{1}{2}} + k_{m-\frac{1}{2}})$$
Из этого сразу следует ответ на последний вопрос: при больших значениях $\frac{\tau}{h^2}$ нарушается устойчивость схемы.
Средние гармонические 3 и 4 не работают в окрестности фронта, так как перед фронтом $u(x,t)=0$ и, следовательно, вычисления могут быть некорректны из-за знаменателя. Более предпочтительным является 1й способ подсчета благодаря его универсальности по сравнению с 3 и 4 и более точному расчету по сравнению со 2.

## 2. Реализация алгоритма расчета

Аппроксимация начальных данных: $$ y_m^0=g(x_m), m = 0,1,..,M $$
Аппроксимация двух краевых условий: $$\alpha_1\frac{y_1^n-y_0^n}{h}+\beta_1y_0^n=f_1(t_n), n = 1,2,..,N $$
$$ \alpha_2\frac{y_M^n-y_{M-1}^n}{h}+\beta_2y_M^n=f_2(t_n), n = 1,2,..,N $$
Здесь для удобства расчетов выполнена замена $-\alpha_1 \rightarrow \alpha_1$.

С учетом разностной схемы $$ \frac{y^{n+1}_m - y^{n}_{m}}{\tau} = \frac{1}{h}(k_{m+\frac{1}{2}}\frac{y^{n+1}_{m+1} - y^{n+1}_{m}}{h} - k_{m-\frac{1}{2}}\frac{y^{n+1}_{m} - y^{n+1}_{m-1}}{h}) $$ на каждом из N слоев мы имеем M+1 уравнение на M+1 неизвестную. Несложно заметить, что матрица такой системы трехдиагональная, следовательно, можно применить метод прогонки.
Опишем принцип расчета в общем случае. Пусть мы привели систему к виду:
$$ Ay=f $$
$$
\left(
\begin{array}{cccc}
c_{0} & -b_{0} & 0 &\ldots & 0\\
-a_{1} & c_{1} & -b_{1} & \ldots & 0\\
\vdots & \vdots & \vdots & \ddots & -b_{M-1}\\
0 & \ldots& 0 & -a_{M} & c_{M}
\end{array}
\right)
\left(
\begin{array}{cccc}
y_{0}\\
y_{1}\\
\vdots\\
y_{M}
\end{array}
\right)
=
\left(
\begin{array}{cccc}
f_{0}\\
f_{1}\\
\vdots\\
f_{M}
\end{array}
\right)
$$

Прямым ходом прогонки вычисляем прогоночные коэффициенты:
$$
\alpha_1 = \frac{b_0}{c_0}\\
\beta_1 = \frac{f_0}{c_0}\\
\alpha_{i+1} = \frac{b_i}{c_i-a_i\alpha_i}, i = 1,2,..,M-1\\
\beta_{i+1} = \frac{f_i+a_i\beta_i}{c_i-a_i\alpha_i}, i = 1,2,..,M\\
$$
Обратным ходом прогонки вычисляем решение:
$$
y_M=\beta_{M+1}\\
y_i=\alpha_{i+1}y_{i+1}+\beta_{i+1}, i=M−1,M−2,..,0
$$

И так решаем систему N раз. Ниже представлены полученные расчетные формулы:

$$
y_0^n(\beta_1-\frac{\alpha_1}{h})+y_1^n\frac{\alpha_1}{h}=f_1(t_n)\\
y_{m-1}^{n+1}\frac{k_{m-\frac{1}{2}}}{h^2}-y_m^{n+1}(\frac{k_{m+\frac{1}{2}}+k_{m-\frac{1}{2}}}{h^2}+\frac{1}{\tau})+y_{m+1}^{n+1}\frac{k_{m+\frac{1}{2}}}{h^2}=-\frac{y_m^n}{\tau}-q(x_m,t_n)\\
y_{M-1}^n(-\frac{\alpha_2}{h})+y_M^n(\frac{\alpha_2}{h}+\beta_2)=f_2(t_n)\\
$$

## 2. Проверка работы алгоритма на задаче с аналитическим решением.

### Задача 1
Рассмотрели следующую задачу с нулевыми краевыми условиями:
$$ u_t=(u^\alpha u_x)_x$$
Аналитическое решение : 
$ \quad u(x,t)=\begin{cases}
(\alpha D)^{\frac{1}{\alpha}}(Dt-x)^{\frac{1}{\alpha}},  \quad {x-Dt \leqslant 0 } \\
0,  \quad {x-Dt > 0 }
\end{cases} $

Результаты расчетов:

<img src='1.png' style='width:1000px;height:600px'>
<img src='2.png' style='width:1000px;height:600px'>

Видно, что 1 и 2 методы расчета подходят для решения этой задачи. Средние гармонические 3 и 4 в знаменателе дают 0, из-за чего вид решения далек от точного. Однако, как мы увидим дальше, для некоторых задач они все же пригодны.

## 3. Исследование сходимости по сетке. Точность решений. Сравнение с аналитическим.

### Задача 2

$$ u_t=(2u u_x)_x+4(\cos x+t)\cos x$$ $$ u(0,t)=1+2t, \quad t\in(0,T)\\ u(X,t)=\cos X + 2t, \quad t\in(0,T)\\ u(x,0)=\cos x, \quad x\in(0,X), X < \frac{\pi}{2} $$
Аналитическое решение: $\quad u(x,t)=\cos x + 2t$

Результаты расчетов:

<img src='3.png' style='width:1000px;height:600px'>
<img src='4.png' style='width:1000px;height:600px'>

### Сходимость по сетке

<img src='5.png'>

### Задача 3

$$ u_t=(u u_x)_x + x^2-6x^2(t+2)^2$$ $$ u(0,t)=0, \quad t\in(0,T)\\ u(X,t)=(t+2)X^2, \quad t\in(0,T)\\ u(x,0)=2 x^2, \quad x\in(0,X) $$
Аналитическое решение: $\quad u(x,t)=(t+2)x^2$

<img src='6.png' style='width:1000px;height:600px'>
<img src='7.png' style='width:1000px;height:600px'>

### Сходимость по сетке

<img src='8.png'>

При правильном выборе шагов по времени и координате все методы сходятся, причем с ростом номера метода его точность убывает. Поэтому оптимальным для расчетов является 1й метод.

## Выводы
+ рассмотренная шеститочечная схема годится для расчета нелинейного уравнения теплопроводности с неоднородной правой частью
+ среди 4х рассмотренных способов расчета коэффициента теплопроводности 1й является наиболее удачным к применению
+ 3й и 4й способы расчета плохо работают в окрестности фронта волны, их применение ограничено
+ нужно грамотно подбирать шаг сетки, иначе возможны отставания фронта волны, нарушение устойчивости схемы

### Литература:

+ Р.П.Федоренко, "Введение в вычислительную физику", 1994
+ А.Д.Полянин, В.Ф.Зайцев, СПРАВОЧНИК ПО НЕЛИНЕЙНЫМ УРАВНЕНИЯМ МАТЕМАТИЧЕСКОЙ ФИЗИКИ: ТОЧНЫЕ РЕШЕНИЯ, 2002
+ М.О.Васильев, "О разностных методах решения нелинейного уравнения теплопроводности. Одномерный случай", 2004